In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

--2023-11-04 13:25:54--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.52.113.109, 2600:1413:b000:78a::317f, 2600:1413:b000:793::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.52.113.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   136MB/s    in 5.1s    

2023-11-04 13:25:59 (155 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [ ]:
!unzip /content/kagglecatsanddogs_5340.zip


In [ ]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt


In [ ]:
input_path = []
label = []
for class_name in os.listdir('PetImages'):
  for path in os.listdir('PetImages/'+class_name):
    if class_name == 'Cat':
      label.append(0)
    else:
      label.append(1)
    input_path.append(os.path.join('PetImages',class_name,path))

print(input_path[5], label[5])

PetImages/Cat/4659.jpg 0


In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac = 1).reset_index(drop = True)
df.head

<bound method NDFrame.head of                         images  label
0       PetImages/Dog/3593.jpg      1
1       PetImages/Dog/3130.jpg      1
2      PetImages/Dog/11759.jpg      1
3       PetImages/Cat/3113.jpg      0
4       PetImages/Dog/8652.jpg      1
...                        ...    ...
24997  PetImages/Cat/11069.jpg      0
24998   PetImages/Dog/3283.jpg      1
24999   PetImages/Cat/1779.jpg      0
25000  PetImages/Cat/12343.jpg      0
25001   PetImages/Cat/6437.jpg      0

[25002 rows x 2 columns]>

In [ ]:
df = df[df['images']!='PetImages/Dog/Thumbs.db']
df = df[df['images']!='PetImages/Cat/Thumbs.db']
df = df[df['images']!='PetImages/Dog/11702.jpg']
df = df[df['images']!='PetImages/Cat/666.jpg']

In [ ]:
df['label'] = df['label'].astype('str')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state =42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255, #normalization
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(train, x_col= 'images', y_col='label',target_size = (112,112), batch_size = 60, class_mode = 'binary')

val_iterator = val_generator.flow_from_dataframe(test, x_col= 'images', y_col='label',target_size = (112,112), batch_size = 60 , class_mode = 'binary')

Found 19998 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [ ]:
model = Sequential([
    Conv2D(96,(11,11),strides = 4, activation = 'relu', input_shape = (112,112,3)),
    MaxPool2D((3,3),strides = 2),
    Conv2D(256,(5,5), padding = 'same', activation = 'relu'),
    MaxPool2D((3,3),strides = 2),
    Conv2D(384,(3,3), padding = 'same', activation = 'relu'),
    Conv2D(384,(3,3), padding = 'same', activation = 'relu'),
    Conv2D(384,(3,3), padding = 'same', activation = 'relu'),
    MaxPool2D((3,3),strides = 2),
    Flatten(),
    Dense(4096, activation = 'relu'),
    Dense(4096, activation = 'relu'),
    Dense(1000, activation = 'relu'),
    Dense(1, activation = 'sigmoid'),
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 26, 26, 96)        34944     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 12, 12, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 12, 12, 256)       614656    
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 5, 5, 256)         0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 5, 5, 384)         885120    
                                                                 
 conv2d_26 (Conv2D)          (None, 5, 5, 384)        

In [ ]:
history = model.fit(train_iterator, epochs = 10, validation_data = val_iterator)

Epoch 1/10
261/334 [======================>.......] - ETA: 20s - loss: 0.6932 - accuracy: 0.5000

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


334/334 [==============================] - 101s 303ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6932 - val_accuracy: 0.4940
Epoch 2/10
137/334 [===========>..................] - ETA: 53s - loss: 0.6932 - accuracy: 0.4960

KeyboardInterrupt: ignored